# Análisis del riesgo de incumplimiento de los prestatarios

Este proyecto consiste en preparar un informe para la división de préstamos de un banco. Se tiene que averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Este informe se tendrá en cuenta para crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

**Propósito del proyecto**

Evaluar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo.

**Hipótesis**

1. Las personas que tienen una mayor cantidad de hijos tienen una tasa de incumplimiento en el pago del préstamo mas alta.
2. El estado civil no afecta al pago a tiempo de sus préstamos.
3. Las personas con ingresos mas altos, gastan mas y por ende no pagan a tiempo sus préstamos.
4. Tener una casa y educarse son los propósitos que afectan al pago a tiempo de los préstamos.

In [ ]:
# Cargar todas las librerías
import pandas as pd
credit = pd.read_csv('/datasets/credit_scoring_eng.csv')

# Carga los datos
print(credit.head(15))


    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  Secondary Education             1   
3          3   -4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -152.779569         50  SECONDARY EDUCATION             1   
8          2   -6929.865299         35    BACHELOR'S DEGREE             0   
9          0   -2188.756445         41  secondary education             1   
10         2   -4171.483647         36    bachelor's degree             0   
11         0    -792.701887         40  secondary education             1   

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo


In [ ]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
rows = len(credit.axes[0])
cols = len(credit.axes[1])

print(f'Número de filas: ' + str(rows))
print(f'Número de columnas: ' + str(cols))



Número de filas: 21525
Número de columnas: 12


In [ ]:
# vamos a mostrar las primeras filas N
print(credit.head(10))


   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024.803754         36  secondary education             1   
2         0   -5623.422610         33  Secondary Education             1   
3         3   -4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   
5         0    -926.185831         27    bachelor's degree             0   
6         0   -2879.202052         43    bachelor's degree             0   
7         0    -152.779569         50  SECONDARY EDUCATION             1   
8         2   -6929.865299         35    BACHELOR'S DEGREE             0   
9         0   -2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

La columna "days_employed" nos muestra números negativos, seguramente fueron errores de digitación o el cálculo de la fórmula estuvo mal, pusieron al revés los datos; adicionalmente, los valores son demasiado altos, en la fila "4" se calcula que el trabajador ha trabajado 945 años. Se necesita investigar esta columna y revisar si es necesario su cambio. En "education" se debe unificar los valores a uno solo "bachelor´s degree" y "secondary education". Igualmente en la columna "purpose" se debe revisar los datos y unficarlos.

In [ ]:
# Obtener información sobre los datos
credit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Las columnas "days_employed" y "total income" tienen valores ausentes.

In [ ]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
print(credit[credit['days_employed'].isna()])



       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65  secondary education             1   
26            0            NaN         41  secondary education             1   
29            0            NaN         63  secondary education             1   
41            0            NaN         50  secondary education             1   
55            0            NaN         54  secondary education             1   
...         ...            ...        ...                  ...           ...   
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
21510         2            NaN         28  secondary education             1   

           family_status  family_status

Los valores ausentes no guardan ninguna correlación con las otras columnas estamos seguros de esto porque al momento de revisar las columnas, cada uno tiene valores y propósitos diferentes.

In [ ]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
credit_1 = pd.read_csv('/datasets/credit_scoring_eng.csv', keep_default_na=False)
credit_1 = credit_1[credit_1['days_employed'] != '']
print(credit_1.head(20))
print()
credit_1['days_employed'].isna().value_counts()


    children        days_employed  dob_years            education  \
0          1   -8437.673027760233         42    bachelor's degree   
1          1   -4024.803753850451         36  secondary education   
2          0   -5623.422610230956         33  Secondary Education   
3          3   -4124.747206540018         32  secondary education   
4          0   340266.07204682194         53  secondary education   
5          0   -926.1858308789183         27    bachelor's degree   
6          0   -2879.202052139952         43    bachelor's degree   
7          0   -152.7795691752705         50  SECONDARY EDUCATION   
8          2   -6929.865298973741         35    BACHELOR'S DEGREE   
9          0  -2188.7564450779378         41  secondary education   
10         2   -4171.483646903305         36    bachelor's degree   
11         0   -792.7018870609315         40  secondary education   
13         0  -1846.6419410560736         54         some college   
14         0  -1844.9561821875545 

False    19351
Name: days_employed, dtype: int64

In [ ]:
credit_1['total_income'].isnull().value_counts()

False    19351
Name: total_income, dtype: int64

**Conclusión intermedia**

El número de filas de la tabla filtrada con valores ausentes es 2174. Los valores ausentes de "days_employed" son los mismos que "total_income".

El porcentaje de valores ausentes es del 10%. No es una porción significativa. Las filas cuyos valores son nulos, no se eliminarán, se sustituirán por la cadena vacía ' '.

In [ ]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
credit_nan = credit[credit['days_employed'].isnull()]
print(credit_nan)


       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65  secondary education             1   
26            0            NaN         41  secondary education             1   
29            0            NaN         63  secondary education             1   
41            0            NaN         50  secondary education             1   
55            0            NaN         54  secondary education             1   
...         ...            ...        ...                  ...           ...   
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
21510         2            NaN         28  secondary education             1   

           family_status  family_status

In [ ]:
# Comprobación de la distribución
print(credit_nan.describe())


          children  days_employed    dob_years  education_id  \
count  2174.000000            0.0  2174.000000   2174.000000   
mean      0.552438            NaN    43.632015      0.800828   
std       1.469356            NaN    12.531481      0.530157   
min      -1.000000            NaN     0.000000      0.000000   
25%       0.000000            NaN    34.000000      0.250000   
50%       0.000000            NaN    43.000000      1.000000   
75%       1.000000            NaN    54.000000      1.000000   
max      20.000000            NaN    73.000000      3.000000   

       family_status_id         debt  total_income  
count       2174.000000  2174.000000           0.0  
mean           0.975161     0.078197           NaN  
std            1.418220     0.268543           NaN  
min            0.000000     0.000000           NaN  
25%            0.000000     0.000000           NaN  
50%            0.000000     0.000000           NaN  
75%            1.000000     0.000000           NaN  

**Posibles razones por las que hay valores ausentes en los datos**

Los valores ausentes pueden deberse a una mala digitación por parte de la persona que estaba llenando los datos. También puede deberse a que el cálculo de estos valores se estaba realizando de una fórmula incorrecta. Los datos ausentes son al azar.


In [ ]:
# Comprobando la distribución en el conjunto de datos entero
print(credit.describe())


           children  days_employed     dob_years  education_id  \
count  21525.000000   19351.000000  21525.000000  21525.000000   
mean       0.538908   63046.497661     43.293380      0.817236   
std        1.381587  140827.311974     12.574584      0.548138   
min       -1.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.423625     33.000000      1.000000   
50%        0.000000   -1203.369529     42.000000      1.000000   
75%        1.000000    -291.095954     53.000000      1.000000   
max       20.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt   total_income  
count      21525.000000  21525.000000   19351.000000  
mean           0.972544      0.080883   26787.568355  
std            1.420324      0.272661   16475.450632  
min            0.000000      0.000000    3306.762000  
25%            0.000000      0.000000   16488.504500  
50%            0.000000      0.000000   23202.870000  
75%            1.000

**Conclusión intermedia**

La distribución en el conjunto de datos original es similar al de la tabla filtrada. Por lo tanto, los valores ausentes en las 2 columnas son aleatorios.

In [ ]:
# Comprobación al eliminar filas con datos ausentes
credit2=credit.dropna()
print(credit2.describe())

           children  days_employed     dob_years  education_id  \
count  19351.000000   19351.000000  19351.000000  19351.000000   
mean       0.537388   63046.497661     43.255336      0.819079   
std        1.371408  140827.311974     12.579170      0.550104   
min       -1.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.423625     33.000000      1.000000   
50%        0.000000   -1203.369529     42.000000      1.000000   
75%        1.000000    -291.095954     53.000000      1.000000   
max       20.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt   total_income  
count      19351.000000  19351.000000   19351.000000  
mean           0.972249      0.081184   26787.568355  
std            1.420596      0.273125   16475.450632  
min            0.000000      0.000000    3306.762000  
25%            0.000000      0.000000   16488.504500  
50%            0.000000      0.000000   23202.870000  
75%            1.000

**Conclusiones**

No se encontraron patrones entre la tabla filtrada y el conjunto de datos original.

Las filas con valores ausentes se se sustituirán por la cadena vacía ' ', porque son el 10% del total de datos, es un porcentaje muy bajo. Al comparar la distribución se llegó a la conclusión de que no afectan al conjunto de datos global. Además, al eliminar las filas con datos ausentes se verificó que la distribución es similar a la distribución del conjunto de datos original, entonces no me afecta si cambio los valores ausentes por una cadena vacia. Se reporta que la columna "days_employed" cuenta con valores negaticos y valores muy altos, por ejemplo hay personas que según los datos llevan trabajando mas de 1000 años, lo cual es ilógico. Se continuará trabajando con estos valores, haciendo los arreglos necesarios para esta columna.

Los pasos siguientes será revisar como se encuentra la información del conjunto de dato y se corregirán los valores como: duplicados, ortografía, mayúsculas o minúsculas, etc. Para continuar con el análisis de datos.

## Transformación de datos


In [ ]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
print(credit['education'].unique())

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [ ]:
# Arregla los registros si es necesario
credit['education'] = credit['education'].str.lower()


In [ ]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
print(credit['education'].unique())


["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


In [ ]:
# Veamos la distribución de los valores en la columna `children`
print(credit['children'].describe())
print()
print(credit[(credit['children'] == -1)].count())


count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

children            47
days_employed       44
dob_years           47
education           47
education_id        47
family_status       47
family_status_id    47
gender              47
income_type         47
debt                47
total_income        44
purpose             47
dtype: int64


In [ ]:
print(credit[(credit['children'] == 20)].count())

children            76
days_employed       67
dob_years           76
education           76
education_id        76
family_status       76
family_status_id    76
gender              76
income_type         76
debt                76
total_income        67
purpose             76
dtype: int64


In [ ]:
med_child = credit['children'].median()
print (med_child)

0.0


El valor mínimo de la columna niños es "-1", esto nos puede representar un problema al momento de realizar otras funciones. El número total de filas con este problema es de "47", que representa el 0.2%. Este valor negativo puede deberse a que las familias tuvieron hijos fallecidos o errores de digitación. De igual forma se tiene 76 personas con una cantidad de 20 hijos (0.35%), lo cual pueda que no sea algo alejado de la realidad, pero los cambiaremos para mejorar nuestros datos. Al tener esta suposición se cambiará el valor "-1" y "20" por "0", esto lo hacemos porque la mediana nos da un valor de "0". Se eligió la mediana porque se tienn valores atípicos como el "-1" y el "20".

In [ ]:
credit['children'] = credit['children'].replace(-1,0) & credit['children'].replace(20,0)


In [ ]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
print(credit['children'].describe())


count    21525.000000
mean         0.470476
std          0.750534
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64


In [ ]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
print(credit['days_employed'].isnull().sum())

errors_days = (2174/21525)*100
print(f'El porcentaje de datos faltantes en days_employed es: ' + str(errors_days) + '%')


2174
El porcentaje de datos faltantes en days_employed es: 10.099883855981417%


A pesar de que es un porcentaje bajo de valores nulos, no se los eliminará, se los cambiará por una cadena vacia y los valores negativos se cambiarán a positivos. Adicional a esto, se encuentran valores cuyas cantidades son ilógicas, para ello, se cambiarán los valores por el tercer cuartil, porque es el 75% de todos los valores. Y el límite máximo de días es 14600 días, porque aquí en el Ecuador es el número de dias en el que la mayor cantidad de personas ha trabajado y se jubila.

In [ ]:
# Aborda los valores problemáticos, si existen.
credit['days_employed'] = credit['days_employed'].abs()

In [ ]:
print(credit['days_employed'].describe())

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64


In [ ]:
print(credit['days_employed'].quantile(.75))

5537.882441480826


In [ ]:
credit.loc[credit['days_employed'] > 14600, 'days_employed'] = 5537.88


In [ ]:
print(credit['days_employed'].describe())

count    19351.000000
mean      2911.026268
std       2389.317980
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.340576
max      14583.152504
Name: days_employed, dtype: float64


In [ ]:
credit['days_employed'] = credit['days_employed'].fillna('')

In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado
print(credit['days_employed'])

0        8437.673028
1        4024.803754
2         5623.42261
3        4124.747207
4            5537.88
            ...     
21520    4529.316663
21521        5537.88
21522    2113.346888
21523    3112.481705
21524    1984.507589
Name: days_employed, Length: 21525, dtype: object


In [ ]:
print(credit['days_employed'].isnull().sum())


0


In [ ]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
print(credit['dob_years'].unique())


[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


In [ ]:
print(credit[credit['dob_years'] == 0].count())

children            101
days_employed       101
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income         91
purpose             101
dtype: int64


En la columna "dob_years" existen 101 personas con edad de "0" (0.46%), son datos que seguramente fueron mal digitados. Debido a que no se tiene registro alguno de estos valores para poder arreglarlos, se recurre a calcular la mediana para cambiar los datos.

In [ ]:
# Resuelve los problemas en la columna `dob_years`, si existen
print (credit['dob_years'].median())

42.0


In [ ]:
credit['dob_years'] = credit['dob_years'].replace(0,42)


In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado
print(credit['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]


In [ ]:
# Veamos los valores de la columna
print(credit['family_status'].unique())


['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


In [ ]:
# Aborda los valores problemáticos en `family_status`, si existen


En la columna "family_status", no existen problemas.

In [ ]:
# Veamos los valores en la columna
print(credit['gender'].unique())

['F' 'M' 'XNA']


In [ ]:
print(credit[credit['gender'] == 'XNA'])

       children days_employed  dob_years     education  education_id  \
10701         0   2358.600502         24  some college             2   

           family_status  family_status_id gender income_type  debt  \
10701  civil partnership                 1    XNA    business     0   

       total_income          purpose  
10701     32624.825  buy real estate  


In [ ]:
# Aborda los valores problemáticos, si existen

La columna "gender" tiene 1 persona con género "XNA", no se tienen registros del significado de este género. Podemos asumir que se trata de una persona LGBTIQ+, por tal motivo no se realizará ninguna acción.

In [ ]:
# Veamos los valores en la columna
print(credit['income_type'].unique())

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']


In [ ]:
# Aborda los valores problemáticos, si existen

En la columna "income_type", no existen problemas.

In [ ]:
# Comprobar los duplicados
print(credit.duplicated().sum())

72


In [ ]:
# Aborda los duplicados, si existen
credit=credit.drop_duplicates().reset_index(drop=True)

In [ ]:
# Última comprobación para ver si tenemos duplicados
print(credit.duplicated().sum())

0


In [ ]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  object 
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21453 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 2.0+ MB


El nuevo conjunto de datos consta de 21470 filas, se eliminaron 55 filas duplicadas, que representaban el 0.25%. Se eliminaron debido a que es un valor muy pequeño dentro de nuestro conjunto de datos.


# Trabajar con valores ausentes

Los diccionarios nos ayudan a almacenar cualquier tipo de información y se puede acceder a la información de manera directa, solo sabiendo la clave. En este caso vamos a trabajar con los diccionarios de "education_id" y "family_dict_id".

In [ ]:
# Encuentra los diccionarios

education_dict = credit.set_index('education_id')['education'].to_dict()

family_dict = credit.set_index('family_status_id')['family_status'].to_dict()

print(education_dict)
print()
print(family_dict)


{0: "bachelor's degree", 1: 'secondary education', 2: 'some college', 3: 'primary education', 4: 'graduate degree'}

{0: 'married', 1: 'civil partnership', 2: 'widow / widower', 3: 'divorced', 4: 'unmarried'}


### Restaurar valores ausentes en `total_income`

Las columnas que tienen valores ausentes son "total income" y "days employed". De las cuales, la columna "days employed" ya se realizó un arreglo en el que se cambiaron las filas vacias por una cadena vacia. Par el caso de la columna "total income" se calculará la media y mediana para ver cual seria la mejor opción para reemplazar en esta columna.

In [ ]:
# Vamos a escribir una función que calcule la categoría de edad
def age_group (dob_years):
    if dob_years <= 26:
        return 'young'
    if dob_years <= 26:
        return 'young'
    if (dob_years > 27) & (dob_years <= 59):
        return 'adult'
    return 'old'

In [ ]:
# Prueba si la función funciona bien
print(age_group(25))
print(age_group(35))
print(age_group(70))

young
adult
old


In [ ]:
# Crear una nueva columna basada en la función
credit['age_group'] = credit['dob_years'].apply(age_group)


In [ ]:
# Comprobar cómo los valores en la nueva columna
print(credit['age_group'].head(10))


0    adult
1    adult
2    adult
3    adult
4    adult
5      old
6    adult
7    adult
8    adult
9    adult
Name: age_group, dtype: object


In [ ]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien

print(credit[credit['total_income'].notnull()].head(10))


   children days_employed  dob_years            education  education_id  \
0         1   8437.673028         42    bachelor's degree             0   
1         1   4024.803754         36  secondary education             1   
2         0    5623.42261         33  secondary education             1   
3         3   4124.747207         32  secondary education             1   
4         0       5537.88         53  secondary education             1   
5         0    926.185831         27    bachelor's degree             0   
6         0   2879.202052         43    bachelor's degree             0   
7         0    152.779569         50  secondary education             1   
8         2   6929.865299         35    bachelor's degree             0   
9         0   2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1           

In [ ]:
# Examina los valores medios de los ingresos en función de los factores que identificaste

credit_family1 = credit.groupby('family_status_id')['total_income'].mean()
credit_education1 = credit.groupby('education_id')['total_income'].mean()

print(credit_family1)
print()
print(credit_education1)


family_status_id
0    27041.784689
1    26694.428597
2    22984.208556
3    27189.354550
4    26934.069805
Name: total_income, dtype: float64

education_id
0    33142.802434
1    24594.503037
2    29045.443644
3    21144.882211
4    27960.024667
Name: total_income, dtype: float64


In [ ]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste

credit_family2 = credit.groupby('family_status_id')['total_income'].median()
credit_education2 = credit.groupby('education_id')['total_income'].median()

print(credit_family2)
print()
print(credit_education2)


family_status_id
0    23389.540
1    23186.534
2    20514.190
3    23515.096
4    23149.028
Name: total_income, dtype: float64

education_id
0    28054.5310
1    21836.5830
2    25618.4640
3    18741.9760
4    25161.5835
Name: total_income, dtype: float64


In [ ]:
credit_pivot_1 = credit.pivot_table(values='total_income', index='age_group', columns='education', aggfunc='median')
print(credit_pivot_1)

education  bachelor's degree  graduate degree  primary education  \
age_group                                                          
adult             28989.7380       25161.5835          19522.738   
old               25904.9790       28334.2150          17370.808   
young             23620.9365              NaN          26303.970   

education  secondary education  some college  
age_group                                     
adult                22320.661     27143.465  
old                  19325.406     26222.172  
young                20718.261     22457.706  


In [ ]:
credit_pivot_2 = credit.pivot_table(values='total_income', index='age_group', columns='income_type', aggfunc='median')
print(credit_pivot_2)

income_type    business  civil servant   employee  entrepreneur  \
age_group                                                         
adult        28085.3745     24123.3445  23044.788           NaN   
old          29235.8765     24887.0735  22994.726     79866.103   
young        23925.6240     22167.7645  20747.911           NaN   

income_type  paternity / maternity leave    retiree   student  unemployed  
age_group                                                                  
adult                           8612.661  19760.454       NaN  21014.3605  
old                                  NaN  18412.925       NaN         NaN  
young                                NaN  14298.976  15712.26         NaN  


In [ ]:
credit_pivot_3 = credit.pivot_table(values='total_income', index='education', columns='income_type', aggfunc='median')
print(credit_pivot_3)

income_type           business  civil servant    employee  entrepreneur  \
education                                                                 
bachelor's degree    32285.664     27601.7775  26502.5190     79866.103   
graduate degree            NaN     17822.7570  31771.3210           NaN   
primary education    21887.825     23734.2870  20159.1860           NaN   
secondary education  25451.310     21864.4750  21848.8175           NaN   
some college         28778.744     25694.7750  24209.4300           NaN   

income_type          paternity / maternity leave    retiree   student  \
education                                                               
bachelor's degree                            NaN  23078.523  15712.26   
graduate degree                              NaN  28334.215       NaN   
primary education                            NaN  16415.785       NaN   
secondary education                     8612.661  18374.857       NaN   
some college                        

La caracteristica que mejor define el ingreso de las personas es "income_type" y "education", porque al tener un mejor nivel de educación como: "bachelor's degree" o "somo college", y al tener un negocio propio o ser emprendedor te dan mejores ingresos. Usaremos la mediana para completar los datos ausentes para devolver la tendencia central porque tenemos algunos valores atípicos como ingresos muy altos o muy bajos dependiendo del caso.

In [ ]:
credit_education2

education_id
0    28054.5310
1    21836.5830
2    25618.4640
3    18741.9760
4    25161.5835
Name: total_income, dtype: float64

In [ ]:
def income_med(row):
    total_inc = row['total_income']
    educ_id = row['education_id']

    if pd.isna(total_inc):
        return credit_education2[educ_id]
    else:
        return total_inc

In [ ]:
# Comprueba si funciona

print(income_med(credit.loc[12]))

21836.583


In [ ]:
# Aplícalo a cada fila
credit['total_income'] = credit.apply(income_med, axis =1)

In [ ]:
# Comprueba si tenemos algún error
print(credit['total_income'].isna().sum())



0


In [ ]:
# Comprobar el número de entradas en las columnas

print(f' El número de filas de total_income es: ' + str(credit['total_income'].shape))
print(f' El número de filas de income_type es: ' + str(credit['income_type'].shape))


 El número de filas de total_income es: (21453,)
 El número de filas de income_type es: (21453,)


###  Restaurar valores en `days_employed`

In [ ]:
credit['days_employed'] = pd.to_numeric(credit['days_employed'], errors = 'ignore')

In [ ]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados
credit_family3 = credit.groupby('family_status_id')['days_employed'].median()
credit_education3 = credit.groupby('education_id')['days_employed'].median()
print(credit_family3)
print()
print(credit_education3)
print()


family_status_id
0    2304.964439
1    1944.345291
2    5537.880000
3    2401.954568
4    1462.009287
Name: days_employed, dtype: float64

education_id
0    1895.747795
1    2392.483500
2    1209.128083
3    3043.933615
4    5444.959090
Name: days_employed, dtype: float64



In [ ]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados
credit_family4 = credit.groupby('family_status_id')['days_employed'].mean()
credit_education4 = credit.groupby('education_id')['days_employed'].mean()
print(credit_family4)
print()
print(credit_education4)
print()


family_status_id
0    2974.090958
1    2752.174329
2    4438.267713
3    3011.063221
4    2301.591922
Name: days_employed, dtype: float64

education_id
0    2629.373886
1    3057.818749
2    1769.211163
3    3222.436386
4    4192.751024
Name: days_employed, dtype: float64



En este caso se va a utilizar la media porque los valores de "days_employed" ya fueron arreglados, entonces ya no hay datos atípicos. Voy a realizar la función con el parámetro family status porque dependiendo de este estado se observa la experiencia laboral que han tenido a lo largo del tiempo.

In [ ]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado

def employ_mean(row):
    days_employed = row['days_employed']
    family_status_id = row['family_status_id']

    if pd.isna(days_employed):
        return credit_family4[family_status_id]
    else:
        return days_employed


In [ ]:
# Comprueba que la función funciona
#la Fila 12 tiene el valor 0 (valor ausente)
print(employ_mean(credit.loc[12]))


2752.174329492404


In [ ]:
# Aplicar la función al income_type
credit_income1 = credit.groupby('income_type')['days_employed'].mean()
print(credit_income1)

income_type
business                       2104.433353
civil servant                  3378.318623
employee                       2315.211789
entrepreneur                    520.848083
paternity / maternity leave    3296.759962
retiree                        5537.880000
student                         578.751554
unemployed                     5537.880000
Name: days_employed, dtype: float64


In [ ]:
def income_mean(row):
    days_employed = row['days_employed']
    income_type = row['income_type']

    if pd.isna(days_employed):
        return credit_income1[income_type]
    else:
        return days_employed

#credit['days_employed'] = credit.apply(employ_mean, axis=1)

In [ ]:
# Comprueba si la función funcionó
print(income_mean(credit.loc[12]))


5537.880000000193


In [ ]:
# Reemplazar valores ausentes
credit['days_employed'] = credit.apply(income_mean, axis=1)


In [ ]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes

print(f' El número de filas de days_employed es: ' + str(credit['days_employed'].shape))
print(f' El número de filas de income_type es: ' + str(credit['income_type'].shape))

 El número de filas de days_employed es: (21453,)
 El número de filas de income_type es: (21453,)


## Clasificación de datos


In [ ]:
# Muestra los valores de los datos seleccionados para la clasificación

data_child = credit[['children', 'debt']]

print(data_child.head(10))

   children  debt
0         1     0
1         1     0
2         0     0
3         3     0
4         0     0
5         0     0
6         0     0
7         0     0
8         2     0
9         0     0


In [ ]:
data_family = credit[['family_status', 'debt']]

print(data_family.head(10))

       family_status  debt
0            married     0
1            married     0
2            married     0
3            married     0
4  civil partnership     0
5  civil partnership     0
6            married     0
7            married     0
8  civil partnership     0
9            married     0


In [ ]:
data_income = credit[['total_income', 'debt']]

print(data_income.head(10))

   total_income  debt
0     40620.102     0
1     17932.802     0
2     23341.752     0
3     42820.568     0
4     25378.572     0
5     40922.170     0
6     38484.156     0
7     21731.829     0
8     15337.093     0
9     23108.150     0


Antes de continuar con la clasificación de los datos, voy a arreglar los datos de la columna "purpose"

In [ ]:
def purpose_property(wrong_values, correct_value):
    for wrong_value in wrong_values:
        credit['purpose'] = credit['purpose'].replace(wrong_values, correct_value)
duplicates1 = ['real estate transactions','buy commercial real estate','housing transactions','buying property for renting out','transactions with commercial real estate','housing','purchase of the house','purchase of the house for my family','construction of own property','property','transactions with my real estate','building a real estate','buy real estate','purchase of my own house','building a property','housing renovation','buy residential real estate']
name1 = 'get property'
purpose_property(duplicates1,name1)
#credit['purpose'] = credit.apply(purpose_property, axis=1)

def purpose_car(wrong_values, correct_value):
    for wrong_value in wrong_values:
        credit['purpose'] = credit['purpose'].replace(wrong_values, correct_value)
duplicates2 = ['car purchase','buying a second-hand car','buying my own car','cars','second-hand car purchase','car','to own a car','purchase of a car','to buy a car']
name2 = 'get a car'
purpose_car(duplicates2,name2)

def purpose_education(wrong_values, correct_value):
    for wrong_value in wrong_values:
        credit['purpose'] = credit['purpose'].replace(wrong_values, correct_value)
duplicates3 = ['supplementary education','education','to become educated','getting an education','to get a supplementary education','getting higher education','profile education','university education','going to university']
name3 = 'get educated'
purpose_education(duplicates3,name3)

def purpose_wedding(wrong_values, correct_value):
    for wrong_value in wrong_values:
        credit['purpose'] = credit['purpose'].replace(wrong_values, correct_value)
duplicates4 = ['to have a wedding','having a wedding','wedding ceremony']
name4 = 'get wedding'
purpose_wedding(duplicates4,name4)

In [ ]:
data_purpose = credit[['purpose', 'debt']]

print(data_purpose.head(10))

        purpose  debt
0  get property     0
1     get a car     0
2  get property     0
3  get educated     0
4   get wedding     0
5  get property     0
6  get property     0
7  get educated     0
8   get wedding     0
9  get property     0


In [ ]:
# Comprobar los valores únicos
data_child_un = data_child['children'].value_counts()
print(data_child_un)

0    14213
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64


In [ ]:
data_family_un = data_family['family_status'].value_counts()
print(data_family_un)

married              12339
civil partnership     4150
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64


In [ ]:
data_income_un = data_income['total_income'].value_counts()
print(data_income_un)

21836.583    1479
28054.531     534
25618.464      70
18741.976      22
42413.096       2
             ... 
27097.085       1
45484.109       1
27715.458       1
23834.534       1
41428.916       1
Name: total_income, Length: 19349, dtype: int64


In [ ]:
data_purpose_un = data_purpose['purpose'].value_counts()
print(data_purpose_un)

get property    10811
get a car        4306
get educated     4013
get wedding      2323
Name: purpose, dtype: int64


Se va a clasificar los datos de los ingresos como: altos ingresos y bajos ingresos. Para ello se va a tomar en cuenta el valor de la canasta básica familiar en el Ecuador. Aquellas personas que superen el valor de la canasta básica familiar del Ecuador se clasifican como "high income" y los que tengan valores inferiores se llamarán "low income". A la fecha el valor de la canasta básica familiar del Ecuador es de: 763 dólares americanos. No tengo datos adicionales de los valores de la columna "total_income", solo me dicen que son ingresos mensuales, pero al ser valores muy altos, asumo que no es el dólar. Pero voy a asumir que son pesos mexicanos. Haciendo la transformación a la fecha de los 763 dólares americanos a pesos mexicanos, me sale que es un valor de 14319 pesos mexicanos.

In [ ]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación

def income_group(row):
    total_income = row['total_income']

    if total_income < 14319:
        return 'low income'
    return 'high income'

credit['income_group'] = credit.apply(income_group, axis=1)

In [ ]:
# Obtener estadísticas resumidas para la columna
credit['income_group'].value_counts()

high income    18152
low income      3301
Name: income_group, dtype: int64

Se escogerá el nivel de ingreso alto "high income" y bajo "low income" para cada propósito. En primera instancia se catalogó como riesgo medio a quienes tienen altos ingresos y buscan una propiedad porque al tener altos ingresos es muy posible que puedan pagar la propiedad tranquilamente. Al tener bajos ingresos y querer pagar una propiedad es de riesgo crítico porque es bastante probable que no la puedan pagar porque no tienen los medios para hacerlo. Tener altos ingresos y querer un auto, es de riesgo medio porque al tener altos ingresos es muy probable que las personas quieren un auto costoso, por tal motivo en algún punto puede ser medio riesgoso llegar a pagar el auto. Al tener bajos ingresos el hecho de querer educarse o casarse es de riesgo medio porque no son valores muy altos, entonces es asequible poder pagar por esos propósitos. Y es mas riesgoso aún para las personas de bajos ingresos querer tener un auto, porque es un valor mas alto que pueden complicarse al momento de querer pagar. Para las personas de altos ingresos es mas seguro querer pagar por los propósitos como educarse o casarse porque no son valores muy altos.

In [ ]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos

def risk (row):
    income_group = row['income_group']
    purpose = row['purpose']

    if income_group == 'high income':
        if purpose == 'get property':
            return 'medium'

    if income_group == 'low income':
        if purpose == 'get property':
            return 'critical'

    if income_group == 'high income':
        if purpose == 'get a car':
            return 'medium'

    if income_group == 'low income':
        if purpose == 'get a car':
            return 'critical'

    if income_group == 'low income':
        if purpose == 'get educated':
            return 'medium'

    if income_group == 'low income':
        if purpose == 'get wedding':
            return 'medium'
    return 'secure'


In [ ]:
# Crear una columna con categorías
credit['risk'] = credit.apply(risk, axis=1)

In [ ]:
# Contar los valores de cada categoría para ver la distribución
credit['risk'].value_counts()

medium      13768
secure       5360
critical     2325
Name: risk, dtype: int64

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [ ]:
# Comprueba los datos sobre los hijos y los pagos puntuales
#1 si presenta deuda

child0 = data_child[data_child['debt'] == 0].value_counts()
print(child0)


children  debt
0         0       13141
1         0        4364
2         0        1858
3         0         303
4         0          37
5         0           9
dtype: int64


In [ ]:
# Calcular la tasa de incumplimiento en función del número de hijos

print('Total de valores para cada número de hijos')
child_total = credit.groupby('children')['debt'].count()
print(child_total)
print()
print('Cantidad de personas que presentan deuda en función del número de hijos')
child_1 = credit.groupby('children')['debt'].sum()
print(child_1)
print()
print('Cantidad de personas que no presentan deuda en función del número de hijos')
child_0 = child_total - child_1
print(child_0)

Total de valores para cada número de hijos
children
0    14213
1     4808
2     2052
3      330
4       41
5        9
Name: debt, dtype: int64

Cantidad de personas que presentan deuda en función del número de hijos
children
0    1072
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

Cantidad de personas que no presentan deuda en función del número de hijos
children
0    13141
1     4364
2     1858
3      303
4       37
5        9
Name: debt, dtype: int64


In [ ]:
print('Tasa de incumplimiento en función del numero de hijos')
print(child_1 / child_0)

Tasa de incumplimiento en función del numero de hijos
children
0    0.081577
1    0.101742
2    0.104413
3    0.089109
4    0.108108
5    0.000000
Name: debt, dtype: float64


**Conclusión**

En función a la última tabla se observa lo siguiente:
1. La cantidad de personas que tiene hijos y tienen deuda no es muy grande, se observa que la gran mayoria paga a tiempo sus deudas.
2. Debido a lo que se mencionó en el punto 1, no hay mucha diferencia en la tasa de incumplimiento entre la cantidad de hijos que las personas puedan tener y que mantienen la deuda. Por lo tanto si existe una ligera correlación entre tener mas hijos y no cumplir con sus deudas, como ocurre en el caso de las personas que tienen 1, 2 y 4 hijos quienes tienen la tasa de incumplimiento mas alta para este caso.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [ ]:
# Comprueba los datos del estado familiar y los pagos a tiempo
print('Total de valores para cada estado civil')
family_total = credit.groupby('family_status')['debt'].count()
print(family_total)
print()
print('Cantidad de personas que presentan deuda en función del estado civil')
family_1 = credit.groupby('family_status')['debt'].sum()
print(family_1)
print()
print('Cantidad de personas que no presentan deuda en función del estado civil')
family_0 = family_total - family_1
print(family_0)


Total de valores para cada estado civil
family_status
civil partnership     4150
divorced              1195
married              12339
unmarried             2810
widow / widower        959
Name: debt, dtype: int64

Cantidad de personas que presentan deuda en función del estado civil
family_status
civil partnership    388
divorced              85
married              931
unmarried            274
widow / widower       63
Name: debt, dtype: int64

Cantidad de personas que no presentan deuda en función del estado civil
family_status
civil partnership     3762
divorced              1110
married              11408
unmarried             2536
widow / widower        896
Name: debt, dtype: int64


In [ ]:
# Calcular la tasa de incumplimiento basada en el estado familiar
print(family_1 / family_0)

family_status
civil partnership    0.103137
divorced             0.076577
married              0.081609
unmarried            0.108044
widow / widower      0.070312
Name: debt, dtype: float64


**Conclusión**

Según lo que se observa en la tasa de incumplimiento basada en el estado familiar, las personas divorciadas y viudas son las que tienen la tasa de incumplimiento mas baja. Posiblemente debido a los problemas que han tenido, han decidido pagar mas rápido sus deudas. A diferencia de las personas que estan bajo unión civíl y no están casadas, quiénes al parecer, al no tener una relación de pareja formal tienden a incumplir con sus deudas.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [ ]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo

print('Total de valores para cada nivel de ingresos')
income_total = credit.groupby('income_group')['debt'].count()
print(income_total)
print()
print('Cantidad de personas que presentan deuda en función del nivel de ingresos')
income_1 = credit.groupby('income_group')['debt'].sum()
print(income_1)
print()
print('Cantidad de personas que no presentan deuda en función del nivel de ingresos')
income_0 = income_total - income_1
print(income_0)
print()
income0 = data_income[data_income['debt'] == 0].value_counts()
print(income0)


Total de valores para cada nivel de ingresos
income_group
high income    18152
low income      3301
Name: debt, dtype: int64

Cantidad de personas que presentan deuda en función del nivel de ingresos
income_group
high income    1486
low income      255
Name: debt, dtype: int64

Cantidad de personas que no presentan deuda en función del nivel de ingresos
income_group
high income    16666
low income      3046
Name: debt, dtype: int64

total_income  debt
21836.583     0       1341
28054.531     0        508
25618.464     0         67
18741.976     0         19
42413.096     0          2
                      ... 
18589.609     0          1
18591.443     0          1
18593.082     0          1
18595.215     0          1
362496.645    0          1
Length: 17778, dtype: int64


In [ ]:
# Calcular la tasa de incumplimiento basada en el nivel de ingresos

print(income_1 / income_0)

income_group
high income    0.089164
low income     0.083716
Name: debt, dtype: float64


**Conclusión**

Como se observa en la tasa de incumplimiento según el nivel de ingresos, no existe relación alguna entre las personas que tienen ingresos mas altos, que las que no lo tienen. Ambos tienden a incumplir con sus deudas de la misma forma.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [ ]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos

print('Total de valores para cada propósito')
purpose_total = credit.groupby('purpose')['debt'].count()
print(purpose_total)
print()
print('Cantidad de personas que presentan deuda en función del propósito')
purpose_1 = credit.groupby('purpose')['debt'].sum()
print(purpose_1)
print()
print('Cantidad de personas que no presentan deuda en función del propósito')
purpose_0 = purpose_total - purpose_1
print(purpose_0)


Total de valores para cada propósito
purpose
get a car        4306
get educated     4013
get property    10811
get wedding      2323
Name: debt, dtype: int64

Cantidad de personas que presentan deuda en función del propósito
purpose
get a car       403
get educated    370
get property    782
get wedding     186
Name: debt, dtype: int64

Cantidad de personas que no presentan deuda en función del propósito
purpose
get a car        3903
get educated     3643
get property    10029
get wedding      2137
Name: debt, dtype: int64


In [ ]:
# Calcular la tasa de incumplimiento basada en el propósito
print(purpose_1 / purpose_0)

purpose
get a car       0.103254
get educated    0.101565
get property    0.077974
get wedding     0.087038
Name: debt, dtype: float64


**Conclusión**

De acuerdo a la tasa de incumplimiento en función del propósito se observa que las personas que quieren comprarse un auto y educarse, son las personas quienes incumplen en mayor medida con el cumplimiento de la deuda. Esto puede deberse a que al comprase un auto solo lo ocupan como medio de transporte y no obtienen un retorno de la inversión, lo mismo pasa al educarse, este propósito no les garantiza un puesto de trabajo o ascenso en su trabajo, por tal razón no tienen un retorno de lo invertido. A diferencia de tener una propiedad, la cual pueden ponerla a trabajar para retornar lo invertido y poder pagar su deuda.

# Conclusión general

1. Se identificaron y completaron los datos
2. La columna days employed y total income presentaron un 10% de datos ausentes que se revisaron y a lo largo del proyecto se fueron llenando estos datos.
3. Las columan "days_employed" presentó datos negativos y muy altos, los cuales se cambiaron a positivos y se estableció un límite para estos datos para reemplazarlos.
4. Se siguieron revisando las demás columnas y no se encontró mayor novedad, se tuvieron que cambiar a minúsculas los datos, en la columna children se tenian datos con -1 y 20, que fueron cambiados a 0, también en la columna dob years se tenian valores iguales a 0 que fueron cambiados por la mediana.
5. La cantidad de número duplicados no fue muy alta, así que se eliminaron
6. La columna total income fue revisada y los valores ausentes se cambiaron por la media en función de la educación.
7. Los valores ausentes de la columna days employed fueron cambiados por a media en función del tipo de ingreso de las personas.
8. Los datos fueron clasificados en función de las hipótesis.
9. Al tener una mayor cantidad de hijos es mas posible no poder pagar el préstamo.
10. El hecho de no tener una relación formal como tener unión civil o no estar casados hace que las personas no trabajen en equipo con su pareja para poder pagar el préstamo.
11. El nivel de ingresos no se relaciona en el hecho de pagar el préstamos.
12. El propósito del préstamo de comprarse un carro y educarse aumenta el riesgo al momento de pagar un préstamo.

